In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Permissão para acessar arquivo do Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#aumentado a capacidade de visualização de colunas e linhas

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows',10000)

In [ ]:
# Carregar os dados
df = pd.read_csv("drive/My Drive/Projeto Futebol/Partidas_Realizadas_PreProcessadas_att.csv")

# Selecionar apenas as colunas numéricas (tipos float e int) do DataFrame
# A correlação mede a relação linear entre as variáveis, variando de -1 (correlação negativa perfeita) a 1 (correlação positiva perfeita)
df_aux = df.select_dtypes(include=[float, int])
df_aux.corr()

,Rodada,Placar_Mandante,Placar_Visitante,chutes_mandante,chutes_visitante,chutes_a_gol_mandante,chutes_a_gol_visitante,posse_de_bola_mandante,posse_de_bola_visitante,passes_mandante,passes_visitante,precisão_de_passe_mandante,precisão_de_passe_visitante,faltas_mandante,faltas_visitante,cartões_amarelos_mandante,cartões_amarelos_visitante,cartões_vermelhos_mandante,cartões_vermelhos_visitante,impedimentos_mandante,impedimentos_visitante,escanteios_mandante,escanteios_visitante,resultado_numerico
Rodada,1.000000,0.048365,-0.053073,0.072613,-0.065395,0.056763,0.006900,0.031709,-0.031709,0.001231,-0.043905,0.035803,-0.041158,-0.021617,-0.089837,-0.114125,-0.074581,0.003847,0.004862,0.016927,-0.006956,0.001863,-0.019824,0.041796
Placar_Mandante,0.048365,1.000000,-0.066361,0.075142,0.002824,0.431913,-0.077771,-0.062647,0.062647,0.001738,0.086077,0.085479,0.222498,0.048584,-0.161072,-0.093120,-0.102084,-0.043634,0.103657,0.017897,-0.012901,-0.035675,0.012758,0.625926
Placar_Visitante,-0.053073,-0.066361,1.000000,-0.018146,0.324182,-0.053096,0.593476,-0.016615,0.016615,-0.031781,0.017748,-0.018133,0.022654,-0.064837,0.023265,0.053043,-0.042555,0.266303,-0.057989,0.027678,-0.002583,-0.027817,0.086950,-0.641258
chutes_mandante,0.072613,0.075142,-0.018146,1.000000,-0.318266,0.558062,-0.217026,0.376501,-0.376501,0.232481,-0.398327,0.178480,-0.404954,-0.022283,-0.113597,-0.127325,0.035344,-0.189963,0.177238,0.022900,-0.098262,0.604910,-0.236533,0.025523
chutes_visitante,-0.065395,0.002824,0.324182,-0.318266,1.000000,-0.147790,0.691972,-0.431704,0.431704,-0.409713,0.321724,-0.417496,0.252551,-0.024618,-0.048110,0.070002,-0.073279,0.262826,-0.234345,-0.144377,-0.029403,-0.292774,0.467063,-0.116392
chutes_a_gol_mandante,0.056763,0.431913,-0.053096,0.558062,-0.147790,1.000000,-0.085225,0.197413,-0.197413,0.173123,-0.173286,0.174295,-0.139579,0.047279,-0.120072,-0.138730,-0.009194,-0.168324,0.170990,0.065020,-0.093809,0.322335,-0.121359,0.270318
chutes_a_gol_visitante,0.006900,-0.077771,0.593476,-0.217026,0.691972,-0.085225,1.000000,-0.259609,0.259609,-0.256097,0.214414,-0.227432,0.146601,-0.020324,-0.037365,0.044750,-0.109456,0.265702,-0.139030,-0.136815,-0.018038,-0.213755,0.312320,-0.357037
posse_de_bola_mandante,0.031709,-0.062647,-0.016615,0.376501,-0.431704,0.197413,-0.259609,1.000000,-1.000000,0.885669,-0.882238,0.703952,-0.681514,-0.082782,0.113389,-0.081862,0.176119,-0.325556,0.134422,0.038424,-0.046574,0.302920,-0.331454,-0.124078
posse_de_bola_visitante,-0.031709,0.062647,0.016615,-0.376501,0.431704,-0.197413,0.259609,-1.000000,1.000000,-0.885669,0.882238,-0.703952,0.681514,0.082782,-0.113389,0.081862,-0.176119,0.325556,-0.134422,-0.038424,0.046574,-0.302920,0.331454,0.124078
passes_mandante,0.001231,0.001738,-0.031781,0.232481,-0.409713,0.173123,-0.256097,0.885669,-0.885669,1.000000,-0.590350,0.796740,-0.393474,-0.146984,-0.000684,-0.138347,0.047199,-0.310803,0.066423,0.049937,0.000966,0.175005,-0.359021,-0.070919


In [ ]:
# Exibir informações sobre o DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Rodada                       300 non-null    int64  
 1   Mandante                     300 non-null    object 
 2   Placar_Mandante              297 non-null    float64
 3   Placar_Visitante             297 non-null    float64
 4   Visitante                    300 non-null    object 
 5   chutes_mandante              297 non-null    float64
 6   chutes_visitante             297 non-null    float64
 7   chutes_a_gol_mandante        297 non-null    float64
 8   chutes_a_gol_visitante       297 non-null    float64
 9   posse_de_bola_mandante       297 non-null    float64
 10  posse_de_bola_visitante      297 non-null    float64
 11  passes_mandante              297 non-null    float64
 12  passes_visitante             297 non-null    float64
 13  precisão_de_passe_ma

In [ ]:
#Verificar a ocorrência de dados duplicados
#Identifica registros duplicados nos dados
dupes=df.duplicated()
sum(dupes)

0

In [ ]:
# Filtrar dados até a última rodada realizada
df_ate_ultima_rodada = df[df['Rodada'] <= 30].copy()

# Exibir uma prévia dos dados filtrados
df_ate_ultima_rodada.tail(10)


,Rodada,Mandante,Placar_Mandante,Placar_Visitante,Visitante,chutes_mandante,chutes_visitante,chutes_a_gol_mandante,chutes_a_gol_visitante,posse_de_bola_mandante,posse_de_bola_visitante,passes_mandante,passes_visitante,precisão_de_passe_mandante,precisão_de_passe_visitante,faltas_mandante,faltas_visitante,cartões_amarelos_mandante,cartões_amarelos_visitante,cartões_vermelhos_mandante,cartões_vermelhos_visitante,impedimentos_mandante,impedimentos_visitante,escanteios_mandante,escanteios_visitante,resultado,resultado_numerico
290,30,Internacional,1.0,0.0,Grêmio,18.0,9.0,6.0,7.0,0.64,0.36,495.0,278.0,0.83,0.67,14.0,15.0,1.0,3.0,0.0,0.0,1.0,1.0,3.0,6.0,VitoriaMandante,1
291,30,Cruzeiro,1.0,1.0,Bahia,11.0,10.0,3.0,4.0,0.48,0.52,402.0,425.0,0.83,0.84,10.0,8.0,1.0,2.0,0.0,0.0,2.0,5.0,5.0,7.0,Empate,0
292,30,Vitória,1.0,0.0,Bragantino,11.0,9.0,3.0,1.0,0.49,0.51,395.0,403.0,0.83,0.80,15.0,8.0,2.0,1.0,0.0,0.0,2.0,0.0,5.0,5.0,VitoriaMandante,1
293,30,Flamengo,0.0,2.0,Fluminense,20.0,12.0,5.0,3.0,0.61,0.39,517.0,350.0,0.84,0.78,15.0,15.0,1.0,3.0,0.0,0.0,3.0,1.0,5.0,1.0,VitoriaVisitante,-1
294,30,Botafogo,1.0,1.0,Criciúma,25.0,6.0,7.0,3.0,0.73,0.27,690.0,258.0,0.87,0.68,12.0,10.0,1.0,2.0,0.0,0.0,1.0,2.0,9.0,1.0,Empate,0
295,30,Corinthians,5.0,2.0,Athletico-PR,14.0,22.0,8.0,6.0,0.61,0.39,565.0,343.0,0.84,0.80,9.0,7.0,4.0,3.0,0.0,0.0,0.0,2.0,3.0,8.0,VitoriaMandante,1
296,30,São Paulo,3.0,0.0,Vasco,14.0,9.0,6.0,3.0,0.48,0.52,415.0,440.0,0.85,0.83,8.0,6.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,5.0,VitoriaMandante,1
297,30,Fortaleza,1.0,1.0,Atlético-MG,14.0,18.0,1.0,5.0,0.46,0.54,380.0,446.0,0.80,0.87,9.0,10.0,2.0,3.0,2.0,0.0,0.0,3.0,7.0,3.0,Empate,0
298,30,Atlético-GO,0.0,0.0,Cuiabá,16.0,6.0,4.0,0.0,0.60,0.40,466.0,324.0,0.85,0.72,17.0,15.0,2.0,3.0,0.0,0.0,0.0,1.0,8.0,4.0,Empate,0
299,30,Juventude,3.0,5.0,Palmeiras,25.0,14.0,6.0,7.0,0.53,0.47,342.0,329.0,0.83,0.80,15.0,19.0,2.0,5.0,0.0,0.0,1.0,5.0,7.0,5.0,VitoriaVisitante,-1


In [ ]:
# Definir as colunas de características (features) e a variável alvo (target)
# As características são todas as colunas, exceto 'Rodada', 'resultado' e 'resultado_numerico'
features = df_ate_ultima_rodada.drop(columns=['Rodada', 'resultado', 'resultado_numerico'])
target = df_ate_ultima_rodada['resultado_numerico']

# Verificar as dimensões das features e do target
print(f"Dimensão das features: {features.shape}")
print(f"Dimensão do target: {target.shape}")


Dimensão das features: (300, 24)
Dimensão do target: (300,)


In [ ]:
# Tratar valores faltantes (NaN) nas features numéricas
imputer = SimpleImputer(strategy='mean')

# Aplicar o imputer nas colunas numéricas
features_imputed = imputer.fit_transform(features.select_dtypes(include=[np.number]))

# Verificar o resultado após o tratamento
print(f"Shape das features após imputação: {features_imputed.shape}")


Shape das features após imputação: (300, 22)


In [ ]:
# Definir as colunas categóricas dos times (Mandante e Visitante)
times_categoricos = ['Mandante', 'Visitante']

# Aplicar a codificação OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
features_encoded = encoder.fit_transform(df_ate_ultima_rodada[times_categoricos]).toarray()

# Verificar o resultado da codificação
print(f"Shape das features categóricas codificadas: {features_encoded.shape}")


Shape das features categóricas codificadas: (300, 40)


In [ ]:
# Combinar as features numéricas com as categóricas codificadas
features_final = np.hstack([features_encoded, features_imputed])

# Verificar o shape das features finais para garantir que estão corretos
print(f"Shape das features finais: {features_final.shape}")


Shape das features finais: (300, 62)


In [ ]:
# Importar bibliotecas para o modelo
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(features_final, target, test_size=0.2, random_state=42)

# Instanciar o modelo RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")


Acurácia do modelo: 88.33%


In [ ]:
# Carregar a tabela Partidas_Nao_Realizadas
df_partidas_restantes = pd.read_csv("drive/My Drive/Projeto Futebol/Partidas_Nao_Realizadas_att.csv")
df_partidas_restantes['Rodada'] = df_partidas_restantes['Rodada'] + 2
df_partidas_restantes = df_partidas_restantes.drop(df_partidas_restantes.index[:10]).reset_index(drop=True)
df_partidas_restantes
#df_partidas_restantes = pd.DataFrame({
#    'Rodada': [30]*10,
#    'Mandante': ['Internacional', 'Cruzeiro', 'Vitória', 'Flamengo', 'Botafogo',
#                 'Corinthians', 'São Paulo', 'Fortaleza', 'Atlético-GO', 'Juventude'],
#    'Visitante': ['Grêmio', 'Bahia', 'Bragantino', 'Fluminense', 'Criciúma',
#                  'Athletico-PR', 'Vasco', 'Atlético-MG', 'Cuiabá', 'Palmeiras'],
#    'Data': ['06/11']*10
#})
#df_partidas_restantes.head(10)

,Rodada,Mandante,Visitante,Data
0,31,Grêmio,Atlético-GO,26/10
1,31,Atlético-MG,Internacional,26/10
2,31,Vitória,Fluminense,26/10
3,31,Flamengo,Juventude,26/10
4,31,Vasco,Bahia,26/10
5,31,Palmeiras,Fortaleza,26/10
6,31,Bragantino,Botafogo,26/10
7,31,Athletico-PR,Cruzeiro,26/10
8,31,Cuiabá,Corinthians,26/10
9,31,Criciúma,São Paulo,26/10


In [ ]:
# Aplicar a mesma transformação OneHotEncoder que você usou nos dados de treinamento
features_rodadas_restantes = encoder.transform(df_partidas_restantes[['Mandante', 'Visitante']]).toarray()

# Verificar o formato das features transformadas
print(f"Shape das features das rodadas restantes: {features_rodadas_restantes.shape}")

Shape das features das rodadas restantes: (80, 40)


In [ ]:
df_ate_ultima_rodada.head()

,Rodada,Mandante,Placar_Mandante,Placar_Visitante,Visitante,chutes_mandante,chutes_visitante,chutes_a_gol_mandante,chutes_a_gol_visitante,posse_de_bola_mandante,posse_de_bola_visitante,passes_mandante,passes_visitante,precisão_de_passe_mandante,precisão_de_passe_visitante,faltas_mandante,faltas_visitante,cartões_amarelos_mandante,cartões_amarelos_visitante,cartões_vermelhos_mandante,cartões_vermelhos_visitante,impedimentos_mandante,impedimentos_visitante,escanteios_mandante,escanteios_visitante,resultado,resultado_numerico
0,1,Vitória,0.0,1.0,Palmeiras,13.0,15.0,5.0,4.0,0.56,0.44,403.0,307.0,0.84,0.78,12.0,15.0,3.0,5.0,0.0,0.0,2.0,3.0,3.0,9.0,VitoriaVisitante,-1
1,1,Internacional,2.0,1.0,Bahia,17.0,9.0,9.0,4.0,0.53,0.47,513.0,467.0,0.85,0.82,21.0,13.0,5.0,3.0,0.0,0.0,3.0,2.0,6.0,3.0,VitoriaMandante,1
2,1,Criciúma,1.0,1.0,Juventude,10.0,15.0,3.0,4.0,0.47,0.53,423.0,464.0,0.80,0.81,11.0,15.0,4.0,6.0,0.0,0.0,3.0,4.0,0.0,4.0,Empate,0
3,1,Fluminense,2.0,2.0,Bragantino,26.0,16.0,11.0,8.0,0.70,0.30,520.0,228.0,0.86,0.68,8.0,19.0,3.0,4.0,0.0,0.0,0.0,0.0,13.0,6.0,Empate,0
4,1,São Paulo,1.0,2.0,Fortaleza,14.0,6.0,5.0,2.0,0.73,0.27,691.0,272.0,0.87,0.67,10.0,14.0,2.0,3.0,0.0,0.0,2.0,1.0,11.0,4.0,VitoriaVisitante,-1


In [ ]:
# Selecionar apenas as colunas numéricas relevantes para calcular a média
colunas_numericas = ['chutes_mandante', 'chutes_visitante', 'posse_de_bola_mandante',
                     'posse_de_bola_visitante']

# Calcular as médias para cada time como mandante
estatisticas_mandante = df_ate_ultima_rodada.groupby('Mandante')[colunas_numericas].mean()

# Verificar o resultado
print(estatisticas_mandante.head())

# Calcular as médias para cada time como visitante
estatisticas_visitante = df_ate_ultima_rodada.groupby('Visitante')[colunas_numericas].mean()

# Verificar o resultado
print(estatisticas_visitante.head())

              chutes_mandante  chutes_visitante  posse_de_bola_mandante  \
Mandante                                                                  
Athletico-PR        20.076923         10.769231                0.545385   
Atlético-GO         14.600000         12.800000                0.465333   
Atlético-MG         14.800000         11.000000                0.591333   
Bahia               13.933333         10.400000                0.558000   
Botafogo            16.466667         10.933333                0.558667   

              posse_de_bola_visitante  
Mandante                               
Athletico-PR                 0.454615  
Atlético-GO                  0.534667  
Atlético-MG                  0.408667  
Bahia                        0.442000  
Botafogo                     0.441333  
              chutes_mandante  chutes_visitante  posse_de_bola_mandante  \
Visitante                                                                 
Athletico-PR        15.875000         12.187

In [ ]:
# Mapear as estatísticas acumuladas para cada confronto da rodada 30
df_partidas_restantes['media_chutes_mandante'] = df_partidas_restantes['Mandante'].map(estatisticas_mandante['chutes_mandante'])
df_partidas_restantes['media_chutes_visitante'] = df_partidas_restantes['Visitante'].map(estatisticas_visitante['chutes_visitante'])
df_partidas_restantes['media_posse_mandante'] = df_partidas_restantes['Mandante'].map(estatisticas_mandante['posse_de_bola_mandante'])
df_partidas_restantes['media_posse_visitante'] = df_partidas_restantes['Visitante'].map(estatisticas_visitante['posse_de_bola_visitante'])


In [ ]:
df_partidas_restantes

,Rodada,Mandante,Visitante,Data,media_chutes_mandante,media_chutes_visitante,media_posse_mandante,media_posse_visitante
0,31,Grêmio,Atlético-GO,26/10,12.866667,13.400000,0.470000,0.460667
1,31,Atlético-MG,Internacional,26/10,14.800000,13.000000,0.591333,0.502000
2,31,Vitória,Fluminense,26/10,12.600000,10.066667,0.426000,0.541333
3,31,Flamengo,Juventude,26/10,15.200000,11.333333,0.558667,0.490667
4,31,Vasco,Bahia,26/10,12.714286,11.866667,0.504286,0.569333
5,31,Palmeiras,Fortaleza,26/10,19.400000,10.714286,0.540000,0.397143
6,31,Bragantino,Botafogo,26/10,14.214286,11.800000,0.457857,0.496667
7,31,Athletico-PR,Cruzeiro,26/10,20.076923,12.133333,0.545385,0.530667
8,31,Cuiabá,Corinthians,26/10,12.066667,12.000000,0.456000,0.464667
9,31,Criciúma,São Paulo,26/10,16.000000,12.500000,0.443333,0.565714


In [ ]:
# Colunas numéricas relevantes para o treinamento do modelo
colunas_numericas = [
    'chutes_mandante', 'chutes_visitante', 'chutes_a_gol_mandante', 'chutes_a_gol_visitante',
    'posse_de_bola_mandante', 'posse_de_bola_visitante', 'passes_mandante', 'passes_visitante',
    'precisão_de_passe_mandante', 'precisão_de_passe_visitante', 'faltas_mandante', 'faltas_visitante',
    'cartões_amarelos_mandante', 'cartões_amarelos_visitante', 'cartões_vermelhos_mandante',
    'cartões_vermelhos_visitante', 'impedimentos_mandante', 'impedimentos_visitante',
    'escanteios_mandante', 'escanteios_visitante'
]

# Colunas como features para o modelo
features_reais = df_ate_ultima_rodada[colunas_numericas]
target = df_ate_ultima_rodada['resultado_numerico']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Dividindo os dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features_reais, target, test_size=0.2, random_state=42)

# Treinar o modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
modelo.fit(X_train, y_train)

# Avaliar o modelo
acuracia = modelo.score(X_test, y_test)
print(f"Acurácia do modelo nos dados de teste: {acuracia * 100:.2f}%")


Acurácia do modelo nos dados de teste: 58.33%


In [ ]:
# Dividir os dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features_reais, target, test_size=0.2, random_state=42)

# Treinar o modelo
modelo_rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, class_weight='balanced')
modelo_rf.fit(X_train, y_train)

# Avaliar o modelo
acuracia_rf = modelo_rf.score(X_test, y_test)
print(f"Acurácia do Random Forest (ajustado): {acuracia_rf * 100:.2f}%")


Acurácia do Random Forest (ajustado): 53.33%


In [ ]:
from sklearn.linear_model import LogisticRegression

# Dividir os dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features_reais, target, test_size=0.2, random_state=42)

# Remover linhas que contenham valores NaN
X_train_sem_nan = X_train.dropna()
y_train_sem_nan = y_train[X_train_sem_nan.index]
X_test_sem_nan = X_test.dropna()
y_test_sem_nan = y_test[X_test_sem_nan.index]

# Treinar o modelo de Regressão Logística
modelo_logistic = LogisticRegression(max_iter=3000, class_weight='balanced')
modelo_logistic.fit(X_train_sem_nan, y_train_sem_nan)

# Avaliar o modelo
acuracia_logistic = modelo_logistic.score(X_test_sem_nan, y_test_sem_nan)
print(f"Acurácia da Regressão Logística: {acuracia_logistic * 100:.2f}%")


Acurácia da Regressão Logística: 68.33%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Definir os hiperparâmetros para testar
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Instanciar o modelo RandomForest
rf = RandomForestClassifier(random_state=42, class_weight='balanced')

# Configurar o GridSearchCV para testar diferentes combinações
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')

# Treinar o modelo
grid_search.fit(X_train_sem_nan, y_train_sem_nan)

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros:", grid_search.best_params_)

# Treinar o modelo com os melhores parâmetros
best_rf_model = grid_search.best_estimator_

# Avaliar o modelo
acuracia_rf = best_rf_model.score(X_test, y_test)
print(f"Acurácia do Random Forest ajustado: {acuracia_rf * 100:.2f}%")

KeyboardInterrupt: 

In [ ]:
from xgboost import XGBClassifier

# Remapear as classes: -1 (Vitória Visitante) -> 0, 0 (Empate) -> 1, 1 (Vitória Mandante) -> 2
y_train_remap = y_train_sem_nan.map({-1: 0, 0: 1, 1: 2})
y_test_remap = y_test.map({-1: 0, 0: 1, 1: 2})

# Instanciar o modelo
modelo_xgb = XGBClassifier(random_state=42)

# Treinar o modelo com as classes remapeadas
modelo_xgb.fit(X_train_sem_nan, y_train_remap)

# Avaliar o modelo com os dados de teste
acuracia_xgb = modelo_xgb.score(X_test, y_test_remap)
print(f"Acurácia do XGBoost: {acuracia_xgb * 100:.2f}%")


Acurácia do XGBoost: 58.33%


In [ ]:
# Calcular a média de chutes para os times como mandantes e visitantes
media_chutes_mandante = df_ate_ultima_rodada.groupby('Mandante')['chutes_mandante'].mean()
media_chutes_visitante = df_ate_ultima_rodada.groupby('Visitante')['chutes_visitante'].mean()

# Calcular a média de chutes a gol
media_chutes_a_gol_mandante = df_ate_ultima_rodada.groupby('Mandante')['chutes_a_gol_mandante'].mean()
media_chutes_a_gol_visitante = df_ate_ultima_rodada.groupby('Visitante')['chutes_a_gol_visitante'].mean()

# Calcular a média de posse de bola
media_posse_mandante = df_ate_ultima_rodada.groupby('Mandante')['posse_de_bola_mandante'].mean()
media_posse_visitante = df_ate_ultima_rodada.groupby('Visitante')['posse_de_bola_visitante'].mean()

# Calcular a média de passes
media_passes_mandante = df_ate_ultima_rodada.groupby('Mandante')['passes_mandante'].mean()
media_passes_visitante = df_ate_ultima_rodada.groupby('Visitante')['passes_visitante'].mean()

# Calcular a média de precisão de passe
media_precisao_passe_mandante = df_ate_ultima_rodada.groupby('Mandante')['precisão_de_passe_mandante'].mean()
media_precisao_passe_visitante = df_ate_ultima_rodada.groupby('Visitante')['precisão_de_passe_visitante'].mean()

# Calcular a média de faltas
media_faltas_mandante = df_ate_ultima_rodada.groupby('Mandante')['faltas_mandante'].mean()
media_faltas_visitante = df_ate_ultima_rodada.groupby('Visitante')['faltas_visitante'].mean()

# Calcular a média de cartões amarelos
media_cartoes_amarelos_mandante = df_ate_ultima_rodada.groupby('Mandante')['cartões_amarelos_mandante'].mean()
media_cartoes_amarelos_visitante = df_ate_ultima_rodada.groupby('Visitante')['cartões_amarelos_visitante'].mean()

# Calcular a média de cartões vermelhos
media_cartoes_vermelhos_mandante = df_ate_ultima_rodada.groupby('Mandante')['cartões_vermelhos_mandante'].mean()
media_cartoes_vermelhos_visitante = df_ate_ultima_rodada.groupby('Visitante')['cartões_vermelhos_visitante'].mean()

# Calcular a média de impedimentos
media_impedimentos_mandante = df_ate_ultima_rodada.groupby('Mandante')['impedimentos_mandante'].mean()
media_impedimentos_visitante = df_ate_ultima_rodada.groupby('Visitante')['impedimentos_visitante'].mean()

# Calcular a média de escanteios
media_escanteios_mandante = df_ate_ultima_rodada.groupby('Mandante')['escanteios_mandante'].mean()
media_escanteios_visitante = df_ate_ultima_rodada.groupby('Visitante')['escanteios_visitante'].mean()

In [ ]:
# Preencher as features das rodadas restantes com as médias acumuladas
df_partidas_restantes['chutes_mandante'] = df_partidas_restantes['Mandante'].map(media_chutes_mandante)
df_partidas_restantes['chutes_visitante'] = df_partidas_restantes['Visitante'].map(media_chutes_visitante)

df_partidas_restantes['chutes_a_gol_mandante'] = df_partidas_restantes['Mandante'].map(media_chutes_a_gol_mandante)
df_partidas_restantes['chutes_a_gol_visitante'] = df_partidas_restantes['Visitante'].map(media_chutes_a_gol_visitante)

df_partidas_restantes['posse_de_bola_mandante'] = df_partidas_restantes['Mandante'].map(media_posse_mandante)
df_partidas_restantes['posse_de_bola_visitante'] = df_partidas_restantes['Visitante'].map(media_posse_visitante)

df_partidas_restantes['passes_mandante'] = df_partidas_restantes['Mandante'].map(media_passes_mandante)
df_partidas_restantes['passes_visitante'] = df_partidas_restantes['Visitante'].map(media_passes_visitante)

df_partidas_restantes['precisao_de_passe_mandante'] = df_partidas_restantes['Mandante'].map(media_precisao_passe_mandante)
df_partidas_restantes['precisao_de_passe_visitante'] = df_partidas_restantes['Visitante'].map(media_precisao_passe_visitante)

df_partidas_restantes['faltas_mandante'] = df_partidas_restantes['Mandante'].map(media_faltas_mandante)
df_partidas_restantes['faltas_visitante'] = df_partidas_restantes['Visitante'].map(media_faltas_visitante)

df_partidas_restantes['cartoes_amarelos_mandante'] = df_partidas_restantes['Mandante'].map(media_cartoes_amarelos_mandante)
df_partidas_restantes['cartoes_amarelos_visitante'] = df_partidas_restantes['Visitante'].map(media_cartoes_amarelos_visitante)

df_partidas_restantes['cartoes_vermelhos_mandante'] = df_partidas_restantes['Mandante'].map(media_cartoes_vermelhos_mandante)
df_partidas_restantes['cartoes_vermelhos_visitante'] = df_partidas_restantes['Visitante'].map(media_cartoes_vermelhos_visitante)

df_partidas_restantes['impedimentos_mandante'] = df_partidas_restantes['Mandante'].map(media_impedimentos_mandante)
df_partidas_restantes['impedimentos_visitante'] = df_partidas_restantes['Visitante'].map(media_impedimentos_visitante)

df_partidas_restantes['escanteios_mandante'] = df_partidas_restantes['Mandante'].map(media_escanteios_mandante)
df_partidas_restantes['escanteios_visitante'] = df_partidas_restantes['Visitante'].map(media_escanteios_visitante)


In [ ]:
# Renomear as colunas de 'media_*' para o formato esperado
df_partidas_restantes.rename(columns={
    'media_chutes_mandante': 'chutes_mandante',
    'media_chutes_visitante': 'chutes_visitante',
    'media_chutes_a_gol_mandante': 'chutes_a_gol_mandante',
    'media_chutes_a_gol_visitante': 'chutes_a_gol_visitante',
    'media_posse_mandante': 'posse_de_bola_mandante',
    'media_posse_visitante': 'posse_de_bola_visitante',
    'media_passes_mandante': 'passes_mandante',
    'media_passes_visitante': 'passes_visitante',
    'media_precisao_de_passe_mandante': 'precisão_de_passe_mandante',
    'media_precisao_de_passe_visitante': 'precisão_de_passe_visitante',
    'media_faltas_mandante': 'faltas_mandante',
    'media_faltas_visitante': 'faltas_visitante',
    'media_cartoes_amarelos_mandante': 'cartões_amarelos_mandante',
    'media_cartoes_amarelos_visitante': 'cartões_amarelos_visitante',
    'media_cartoes_vermelhos_mandante': 'cartões_vermelhos_mandante',
    'media_cartoes_vermelhos_visitante': 'cartões_vermelhos_visitante',
    'media_impedimentos_mandante': 'impedimentos_mandante',
    'media_impedimentos_visitante': 'impedimentos_visitante',
    'media_escanteios_mandante': 'escanteios_mandante',
    'media_escanteios_visitante': 'escanteios_visitante'
}, inplace=True)

# Renomear as colunas para que correspondam aos nomes usados no treinamento
df_partidas_restantes.rename(columns={
    'cartoes_amarelos_mandante': 'cartões_amarelos_mandante',
    'cartoes_amarelos_visitante': 'cartões_amarelos_visitante',
    'cartoes_vermelhos_mandante': 'cartões_vermelhos_mandante',
    'cartoes_vermelhos_visitante': 'cartões_vermelhos_visitante',
    'precisao_de_passe_mandante': 'precisão_de_passe_mandante',
    'precisao_de_passe_visitante': 'precisão_de_passe_visitante'
}, inplace=True)

# Verificar a ordem das features no modelo treinado
print(modelo_rf.feature_names_in_)

['chutes_mandante' 'chutes_visitante' 'chutes_a_gol_mandante'
 'chutes_a_gol_visitante' 'posse_de_bola_mandante'
 'posse_de_bola_visitante' 'passes_mandante' 'passes_visitante'
 'precisão_de_passe_mandante' 'precisão_de_passe_visitante'
 'faltas_mandante' 'faltas_visitante' 'cartões_amarelos_mandante'
 'cartões_amarelos_visitante' 'cartões_vermelhos_mandante'
 'cartões_vermelhos_visitante' 'impedimentos_mandante'
 'impedimentos_visitante' 'escanteios_mandante' 'escanteios_visitante']


In [ ]:
# Reordenar as colunas de acordo com a ordem do treinamento
features_rodadas_restantes = df_partidas_restantes[modelo_rf.feature_names_in_]
features_rodadas_restantes.head()

,chutes_mandante,chutes_mandante,chutes_visitante,chutes_visitante,chutes_a_gol_mandante,chutes_a_gol_visitante,posse_de_bola_mandante,posse_de_bola_mandante,posse_de_bola_visitante,posse_de_bola_visitante,passes_mandante,passes_visitante,precisão_de_passe_mandante,precisão_de_passe_visitante,faltas_mandante,faltas_visitante,cartões_amarelos_mandante,cartões_amarelos_visitante,cartões_vermelhos_mandante,cartões_vermelhos_visitante,impedimentos_mandante,impedimentos_visitante,escanteios_mandante,escanteios_visitante
0,12.866667,12.866667,13.400000,13.400000,4.400000,4.466667,0.470000,0.470000,0.460667,0.460667,407.000000,404.200000,0.784667,0.798000,10.533333,11.400000,2.266667,2.800000,0.200000,0.066667,0.533333,1.600000,5.533333,4.333333
1,14.800000,14.800000,13.000000,13.000000,4.466667,4.066667,0.591333,0.591333,0.502000,0.502000,513.333333,411.466667,0.852000,0.795333,13.733333,15.133333,2.466667,3.266667,0.333333,0.066667,1.533333,1.533333,4.533333,4.066667
2,12.600000,12.600000,10.066667,10.066667,4.000000,3.266667,0.426000,0.426000,0.541333,0.541333,355.666667,472.133333,0.796000,0.839333,11.400000,12.333333,2.133333,2.533333,0.266667,0.200000,1.466667,0.733333,4.333333,3.666667
3,15.200000,15.200000,11.333333,11.333333,5.533333,3.600000,0.558667,0.558667,0.490667,0.490667,489.400000,390.533333,0.844667,0.796667,11.600000,12.866667,1.400000,3.133333,0.000000,0.466667,1.466667,1.533333,6.466667,4.000000
4,12.714286,12.714286,11.866667,11.866667,4.285714,4.266667,0.504286,0.504286,0.569333,0.569333,419.857143,530.066667,0.817143,0.860000,14.142857,10.466667,2.714286,2.200000,0.142857,0.000000,1.285714,1.800000,4.000000,4.200000


In [ ]:
df_partidas_restantes.drop(columns=['Data'], inplace=True)
df_partidas_restantes.head()

,Rodada,Mandante,Visitante,chutes_mandante,chutes_visitante,posse_de_bola_mandante,posse_de_bola_visitante,chutes_mandante,chutes_visitante,chutes_a_gol_mandante,chutes_a_gol_visitante,posse_de_bola_mandante,posse_de_bola_visitante,passes_mandante,passes_visitante,precisão_de_passe_mandante,precisão_de_passe_visitante,faltas_mandante,faltas_visitante,cartões_amarelos_mandante,cartões_amarelos_visitante,cartões_vermelhos_mandante,cartões_vermelhos_visitante,impedimentos_mandante,impedimentos_visitante,escanteios_mandante,escanteios_visitante
0,31,Grêmio,Atlético-GO,12.866667,13.400000,0.470000,0.460667,12.866667,13.400000,4.400000,4.466667,0.470000,0.460667,407.000000,404.200000,0.784667,0.798000,10.533333,11.400000,2.266667,2.800000,0.200000,0.066667,0.533333,1.600000,5.533333,4.333333
1,31,Atlético-MG,Internacional,14.800000,13.000000,0.591333,0.502000,14.800000,13.000000,4.466667,4.066667,0.591333,0.502000,513.333333,411.466667,0.852000,0.795333,13.733333,15.133333,2.466667,3.266667,0.333333,0.066667,1.533333,1.533333,4.533333,4.066667
2,31,Vitória,Fluminense,12.600000,10.066667,0.426000,0.541333,12.600000,10.066667,4.000000,3.266667,0.426000,0.541333,355.666667,472.133333,0.796000,0.839333,11.400000,12.333333,2.133333,2.533333,0.266667,0.200000,1.466667,0.733333,4.333333,3.666667
3,31,Flamengo,Juventude,15.200000,11.333333,0.558667,0.490667,15.200000,11.333333,5.533333,3.600000,0.558667,0.490667,489.400000,390.533333,0.844667,0.796667,11.600000,12.866667,1.400000,3.133333,0.000000,0.466667,1.466667,1.533333,6.466667,4.000000
4,31,Vasco,Bahia,12.714286,11.866667,0.504286,0.569333,12.714286,11.866667,4.285714,4.266667,0.504286,0.569333,419.857143,530.066667,0.817143,0.860000,14.142857,10.466667,2.714286,2.200000,0.142857,0.000000,1.285714,1.800000,4.000000,4.200000


In [ ]:
# Verificar as colunas de ambas as tabelas
print(df_ate_ultima_rodada.columns)
print(df_partidas_restantes.columns)
# Remover colunas duplicadas em partidas_rodada_28
df_partidas_restantes = df_partidas_restantes.loc[:, ~df_partidas_restantes.columns.duplicated()]

Index(['Rodada', 'Mandante', 'Placar_Mandante', 'Placar_Visitante',
       'Visitante', 'chutes_mandante', 'chutes_visitante',
       'chutes_a_gol_mandante', 'chutes_a_gol_visitante',
       'posse_de_bola_mandante', 'posse_de_bola_visitante', 'passes_mandante',
       'passes_visitante', 'precisão_de_passe_mandante',
       'precisão_de_passe_visitante', 'faltas_mandante', 'faltas_visitante',
       'cartões_amarelos_mandante', 'cartões_amarelos_visitante',
       'cartões_vermelhos_mandante', 'cartões_vermelhos_visitante',
       'impedimentos_mandante', 'impedimentos_visitante',
       'escanteios_mandante', 'escanteios_visitante', 'resultado',
       'resultado_numerico'],
      dtype='object')
Index(['Rodada', 'Mandante', 'Visitante', 'chutes_mandante',
       'chutes_visitante', 'posse_de_bola_mandante', 'posse_de_bola_visitante',
       'chutes_mandante', 'chutes_visitante', 'chutes_a_gol_mandante',
       'chutes_a_gol_visitante', 'posse_de_bola_mandante',
       'posse_de_bol

In [ ]:
# Concatenar as tabelas e preencher colunas faltantes com NaN
partidas_total = pd.concat([df_ate_ultima_rodada, df_partidas_restantes], ignore_index=True)


In [ ]:
partidas_total.tail()

,Rodada,Mandante,Placar_Mandante,Placar_Visitante,Visitante,chutes_mandante,chutes_visitante,chutes_a_gol_mandante,chutes_a_gol_visitante,posse_de_bola_mandante,posse_de_bola_visitante,passes_mandante,passes_visitante,precisão_de_passe_mandante,precisão_de_passe_visitante,faltas_mandante,faltas_visitante,cartões_amarelos_mandante,cartões_amarelos_visitante,cartões_vermelhos_mandante,cartões_vermelhos_visitante,impedimentos_mandante,impedimentos_visitante,escanteios_mandante,escanteios_visitante,resultado,resultado_numerico
375,38,Palmeiras,NaN,NaN,Fluminense,19.400000,10.066667,6.200000,3.266667,0.540000,0.541333,409.466667,472.133333,0.810667,0.839333,14.466667,12.333333,2.200000,2.533333,0.200000,0.200000,2.200000,0.733333,7.600000,3.666667,NaN,NaN
376,38,Bragantino,NaN,NaN,Criciúma,14.214286,11.933333,4.571429,3.933333,0.457857,0.426667,362.571429,378.933333,0.760000,0.800000,13.071429,13.266667,3.071429,2.466667,0.285714,0.066667,1.142857,1.733333,4.428571,3.533333,NaN,NaN
377,38,Fortaleza,NaN,NaN,Internacional,13.687500,13.000000,4.437500,4.066667,0.446875,0.502000,370.562500,411.466667,0.794375,0.795333,14.312500,15.133333,2.437500,3.266667,0.312500,0.066667,1.375000,1.533333,5.562500,4.066667,NaN,NaN
378,38,Cuiabá,NaN,NaN,Vasco,12.066667,10.733333,3.733333,3.200000,0.456000,0.453333,391.600000,373.733333,0.819333,0.778667,12.533333,11.066667,2.400000,2.200000,0.066667,0.266667,1.666667,1.866667,4.733333,4.666667,NaN,NaN
379,38,Juventude,NaN,NaN,Cruzeiro,13.533333,12.133333,4.866667,4.400000,0.474667,0.530667,391.200000,461.000000,0.800000,0.825333,12.133333,12.333333,2.066667,2.466667,0.200000,0.266667,1.200000,1.533333,4.800000,4.266667,NaN,NaN


In [ ]:
# Separar as últimas linhas que foram adicionadas
df_partidas_restantes_separada = partidas_total.iloc[-80:]
df_partidas_restantes_separada

,Rodada,Mandante,Placar_Mandante,Placar_Visitante,Visitante,chutes_mandante,chutes_visitante,chutes_a_gol_mandante,chutes_a_gol_visitante,posse_de_bola_mandante,posse_de_bola_visitante,passes_mandante,passes_visitante,precisão_de_passe_mandante,precisão_de_passe_visitante,faltas_mandante,faltas_visitante,cartões_amarelos_mandante,cartões_amarelos_visitante,cartões_vermelhos_mandante,cartões_vermelhos_visitante,impedimentos_mandante,impedimentos_visitante,escanteios_mandante,escanteios_visitante,resultado,resultado_numerico
300,31,Grêmio,NaN,NaN,Atlético-GO,12.866667,13.400000,4.400000,4.466667,0.470000,0.460667,407.000000,404.200000,0.784667,0.798000,10.533333,11.400000,2.266667,2.800000,0.200000,0.066667,0.533333,1.600000,5.533333,4.333333,NaN,NaN
301,31,Atlético-MG,NaN,NaN,Internacional,14.800000,13.000000,4.466667,4.066667,0.591333,0.502000,513.333333,411.466667,0.852000,0.795333,13.733333,15.133333,2.466667,3.266667,0.333333,0.066667,1.533333,1.533333,4.533333,4.066667,NaN,NaN
302,31,Vitória,NaN,NaN,Fluminense,12.600000,10.066667,4.000000,3.266667,0.426000,0.541333,355.666667,472.133333,0.796000,0.839333,11.400000,12.333333,2.133333,2.533333,0.266667,0.200000,1.466667,0.733333,4.333333,3.666667,NaN,NaN
303,31,Flamengo,NaN,NaN,Juventude,15.200000,11.333333,5.533333,3.600000,0.558667,0.490667,489.400000,390.533333,0.844667,0.796667,11.600000,12.866667,1.400000,3.133333,0.000000,0.466667,1.466667,1.533333,6.466667,4.000000,NaN,NaN
304,31,Vasco,NaN,NaN,Bahia,12.714286,11.866667,4.285714,4.266667,0.504286,0.569333,419.857143,530.066667,0.817143,0.860000,14.142857,10.466667,2.714286,2.200000,0.142857,0.000000,1.285714,1.800000,4.000000,4.200000,NaN,NaN
305,31,Palmeiras,NaN,NaN,Fortaleza,19.400000,10.714286,6.200000,3.142857,0.540000,0.397143,409.466667,349.142857,0.810667,0.754286,14.466667,11.428571,2.200000,2.928571,0.200000,0.357143,2.200000,1.714286,7.600000,3.785714,NaN,NaN
306,31,Bragantino,NaN,NaN,Botafogo,14.214286,11.800000,4.571429,4.866667,0.457857,0.496667,362.571429,413.333333,0.760000,0.817333,13.071429,17.066667,3.071429,3.466667,0.285714,0.133333,1.142857,1.400000,4.428571,5.066667,NaN,NaN
307,31,Athletico-PR,NaN,NaN,Cruzeiro,20.076923,12.133333,6.076923,4.400000,0.545385,0.530667,462.153846,461.000000,0.836154,0.825333,13.153846,12.333333,2.076923,2.466667,0.230769,0.266667,1.307692,1.533333,7.538462,4.266667,NaN,NaN
308,31,Cuiabá,NaN,NaN,Corinthians,12.066667,12.000000,3.733333,3.200000,0.456000,0.464667,391.600000,364.200000,0.819333,0.792667,12.533333,13.133333,2.400000,2.600000,0.066667,0.200000,1.666667,1.733333,4.733333,4.600000,NaN,NaN
309,31,Criciúma,NaN,NaN,São Paulo,16.000000,12.500000,5.266667,4.285714,0.443333,0.565714,370.066667,493.714286,0.764000,0.854286,13.533333,13.071429,2.666667,2.642857,0.200000,0.214286,2.133333,1.428571,5.533333,4.928571,NaN,NaN


In [ ]:
# Verificar as features esperadas pelo modelo
print(modelo_rf.feature_names_in_)
# Verificar as colunas no DataFrame das últimas 10 linhas
print(df_partidas_restantes_separada.columns)


['chutes_mandante' 'chutes_visitante' 'chutes_a_gol_mandante'
 'chutes_a_gol_visitante' 'posse_de_bola_mandante'
 'posse_de_bola_visitante' 'passes_mandante' 'passes_visitante'
 'precisão_de_passe_mandante' 'precisão_de_passe_visitante'
 'faltas_mandante' 'faltas_visitante' 'cartões_amarelos_mandante'
 'cartões_amarelos_visitante' 'cartões_vermelhos_mandante'
 'cartões_vermelhos_visitante' 'impedimentos_mandante'
 'impedimentos_visitante' 'escanteios_mandante' 'escanteios_visitante']
Index(['Rodada', 'Mandante', 'Placar_Mandante', 'Placar_Visitante',
       'Visitante', 'chutes_mandante', 'chutes_visitante',
       'chutes_a_gol_mandante', 'chutes_a_gol_visitante',
       'posse_de_bola_mandante', 'posse_de_bola_visitante', 'passes_mandante',
       'passes_visitante', 'precisão_de_passe_mandante',
       'precisão_de_passe_visitante', 'faltas_mandante', 'faltas_visitante',
       'cartões_amarelos_mandante', 'cartões_amarelos_visitante',
       'cartões_vermelhos_mandante', 'cartões_v

In [ ]:
# Fazer a previsão das probabilidades para as últimas 10 linhas (rodada 30)
probabilidades_rodada_28 = modelo_rf.predict_proba(df_partidas_restantes_separada[modelo_rf.feature_names_in_])

# Exibir as probabilidades para cada confronto com o nome dos times
for i, prob in enumerate(probabilidades_rodada_28):
    mandante = df_partidas_restantes_separada.iloc[i]['Mandante']
    visitante = df_partidas_restantes_separada.iloc[i]['Visitante']
    print(f"Confronto {i+1} - {mandante} x {visitante}:")
    print(f"Probabilidade de Vitória do Visitante: {prob[0]:.2f}")
    print(f"Probabilidade de Empate: {prob[1]:.2f}")
    print(f"Probabilidade de Vitória do Mandante: {prob[2]:.2f}\n")


Confronto 1 - Internacional x Grêmio:
Probabilidade de Vitória do Visitante: 0.22
Probabilidade de Empate: 0.19
Probabilidade de Vitória do Mandante: 0.59

Confronto 2 - Cruzeiro x Bahia:
Probabilidade de Vitória do Visitante: 0.38
Probabilidade de Empate: 0.15
Probabilidade de Vitória do Mandante: 0.47

Confronto 3 - Vitória x Bragantino:
Probabilidade de Vitória do Visitante: 0.27
Probabilidade de Empate: 0.25
Probabilidade de Vitória do Mandante: 0.48

Confronto 4 - Flamengo x Fluminense:
Probabilidade de Vitória do Visitante: 0.12
Probabilidade de Empate: 0.06
Probabilidade de Vitória do Mandante: 0.83

Confronto 5 - Botafogo x Criciúma:
Probabilidade de Vitória do Visitante: 0.25
Probabilidade de Empate: 0.19
Probabilidade de Vitória do Mandante: 0.56

Confronto 6 - Corinthians x Athletico-PR:
Probabilidade de Vitória do Visitante: 0.27
Probabilidade de Empate: 0.20
Probabilidade de Vitória do Mandante: 0.52

Confronto 7 - São Paulo x Vasco:
Probabilidade de Vitória do Visitante: 

In [ ]:
# Fazer a previsão das probabilidades para a rodada 30 usando o modelo de Regressão Logística
probabilidades_rodada_30 = modelo_logistic.predict_proba(df_partidas_restantes_separada[modelo_logistic.feature_names_in_])

# Exibir as probabilidades para cada confronto com o nome dos times
for i, prob in enumerate(probabilidades_rodada_30):
    mandante = df_partidas_restantes_separada.iloc[i]['Mandante']
    visitante = df_partidas_restantes_separada.iloc[i]['Visitante']
    print(f"Confronto {i+1} - {mandante} x {visitante}:")
    print(f"Probabilidade de Vitória do Visitante: {prob[0]:.2f}")
    print(f"Probabilidade de Empate: {prob[1]:.2f}")
    print(f"Probabilidade de Vitória do Mandante: {prob[2]:.2f}\n")


Confronto 1 - Internacional x Grêmio:
Probabilidade de Vitória do Visitante: 0.27
Probabilidade de Empate: 0.29
Probabilidade de Vitória do Mandante: 0.44

Confronto 2 - Cruzeiro x Bahia:
Probabilidade de Vitória do Visitante: 0.29
Probabilidade de Empate: 0.19
Probabilidade de Vitória do Mandante: 0.51

Confronto 3 - Vitória x Bragantino:
Probabilidade de Vitória do Visitante: 0.38
Probabilidade de Empate: 0.45
Probabilidade de Vitória do Mandante: 0.17

Confronto 4 - Flamengo x Fluminense:
Probabilidade de Vitória do Visitante: 0.20
Probabilidade de Empate: 0.14
Probabilidade de Vitória do Mandante: 0.66

Confronto 5 - Botafogo x Criciúma:
Probabilidade de Vitória do Visitante: 0.22
Probabilidade de Empate: 0.40
Probabilidade de Vitória do Mandante: 0.38

Confronto 6 - Corinthians x Athletico-PR:
Probabilidade de Vitória do Visitante: 0.23
Probabilidade de Empate: 0.39
Probabilidade de Vitória do Mandante: 0.38

Confronto 7 - São Paulo x Vasco:
Probabilidade de Vitória do Visitante: 

In [ ]:
# RandomForest
# Criar um DataFrame para armazenar as probabilidades dos jogos
probabilidades_jogos_restantes_RF = pd.DataFrame(columns=['Rodada', 'Mandante', 'Visitante', 'Prob_Vit_Visitante', 'Prob_Empate', 'Prob_Vit_Mandante'])

# Iterar sobre cada jogo nas rodadas restantes e prever as probabilidades
for index, row in df_partidas_restantes.iterrows():
    # Extrair as features (estatísticas do jogo) para passar ao modelo
    features_jogo = row[modelo_rf.feature_names_in_]

    # Fazer a previsão das probabilidades para esse jogo
    probabilidades = modelo_rf.predict_proba([features_jogo])[0]

    # Armazenar as probabilidades no DataFrame
    probabilidades_jogos_restantes_RF = pd.concat([probabilidades_jogos_restantes_RF, pd.DataFrame({
        'Rodada': [row['Rodada']],
        'Mandante': [row['Mandante']],
        'Visitante': [row['Visitante']],
        'Prob_Vit_Visitante': [probabilidades[0]],
        'Prob_Empate': [probabilidades[1]],
        'Prob_Vit_Mandante': [probabilidades[2]]
    })], ignore_index=True)

# Exibir as probabilidades previstas para os jogos restantes
probabilidades_jogos_restantes_RF

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
<ipython-input-50-4a8d7196bad6>:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  probabilidades_jogos_restantes_RF = pd.concat([probabilidades_jogos_restantes_RF, pd.DataFrame({
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/p

,Rodada,Mandante,Visitante,Prob_Vit_Visitante,Prob_Empate,Prob_Vit_Mandante
0,31,Grêmio,Atlético-GO,0.293931,0.367954,0.338115
1,31,Atlético-MG,Internacional,0.274296,0.250597,0.475108
2,31,Vitória,Fluminense,0.113319,0.141218,0.745463
3,31,Flamengo,Juventude,0.262594,0.223582,0.513824
4,31,Vasco,Bahia,0.294023,0.251608,0.454368
5,31,Palmeiras,Fortaleza,0.383538,0.216439,0.400023
6,31,Bragantino,Botafogo,0.224222,0.373415,0.402363
7,31,Athletico-PR,Cruzeiro,0.383413,0.170945,0.445643
8,31,Cuiabá,Corinthians,0.242174,0.211024,0.546802
9,31,Criciúma,São Paulo,0.152583,0.317932,0.529485


In [ ]:
# Reordenar as colunas
probabilidades_jogos_restantes_RF = probabilidades_jogos_restantes_RF.reindex(columns=['Rodada', 'Mandante', 'Visitante', 'Prob_Vit_Mandante', 'Prob_Empate', 'Prob_Vit_Visitante'])
probabilidades_jogos_restantes_RF.head()

,Rodada,Mandante,Visitante,Prob_Vit_Mandante,Prob_Empate,Prob_Vit_Visitante
0,31,Grêmio,Atlético-GO,0.338115,0.367954,0.293931
1,31,Atlético-MG,Internacional,0.475108,0.250597,0.274296
2,31,Vitória,Fluminense,0.745463,0.141218,0.113319
3,31,Flamengo,Juventude,0.513824,0.223582,0.262594
4,31,Vasco,Bahia,0.454368,0.251608,0.294023


In [ ]:
probabilidades_jogos_restantes_RF.to_csv("drive/My Drive/Projeto Futebol/probabilidades_jogos_restantes_RandomForest_rodada30.csv", index=False)

In [ ]:
# Criar um DataFrame para armazenar as probabilidades dos jogos
probabilidades_jogos_restantes_RL = pd.DataFrame(columns=['Rodada', 'Mandante', 'Visitante', 'Prob_Vit_Visitante', 'Prob_Empate', 'Prob_Vit_Mandante'])

# Iterar sobre cada jogo nas rodadas restantes e prever as probabilidades
for index, row in df_partidas_restantes.iterrows():
    # Extrair as features (estatísticas do jogo) para passar ao modelo
    features_jogo = row[modelo_logistic.feature_names_in_].values.reshape(1, -1)  # Certificar que os valores estão em forma de array 2D

    # Fazer a previsão das probabilidades para esse jogo (remova os colchetes extras)
    probabilidades = modelo_logistic.predict_proba(features_jogo)[0]

    # Armazenar as probabilidades no DataFrame
    probabilidades_jogos_restantes_RL = pd.concat([probabilidades_jogos_restantes_RL, pd.DataFrame({
        'Rodada': [row['Rodada']],
        'Mandante': [row['Mandante']],
        'Visitante': [row['Visitante']],
        'Prob_Vit_Visitante': [probabilidades[0]],
        'Prob_Empate': [probabilidades[1]],
        'Prob_Vit_Mandante': [probabilidades[2]]
    })], ignore_index=True)

# Exibir as probabilidades previstas para os jogos restantes
probabilidades_jogos_restantes_RL


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
<ipython-input-53-512792bc4df0>:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  probabilidades_jogos_restantes_RL = pd.concat([probabilidades_jogos_restantes_RL, pd.DataFrame({
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/di

,Rodada,Mandante,Visitante,Prob_Vit_Visitante,Prob_Empate,Prob_Vit_Mandante
0,31,Grêmio,Atlético-GO,0.374581,0.336725,0.288695
1,31,Atlético-MG,Internacional,0.385107,0.341815,0.273077
2,31,Vitória,Fluminense,0.231674,0.232638,0.535688
3,31,Flamengo,Juventude,0.244867,0.329921,0.425212
4,31,Vasco,Bahia,0.256763,0.209729,0.533508
5,31,Palmeiras,Fortaleza,0.219371,0.467362,0.313267
6,31,Bragantino,Botafogo,0.356334,0.448979,0.194687
7,31,Athletico-PR,Cruzeiro,0.314854,0.393061,0.292085
8,31,Cuiabá,Corinthians,0.252049,0.398703,0.349248
9,31,Criciúma,São Paulo,0.237780,0.372418,0.389802


In [ ]:
# Reordenar as colunas
probabilidades_jogos_restantes_RL = probabilidades_jogos_restantes_RL.reindex(columns=['Rodada', 'Mandante', 'Visitante', 'Prob_Vit_Mandante', 'Prob_Empate', 'Prob_Vit_Visitante'])
probabilidades_jogos_restantes_RL.head()

,Rodada,Mandante,Visitante,Prob_Vit_Mandante,Prob_Empate,Prob_Vit_Visitante
0,31,Grêmio,Atlético-GO,0.288695,0.336725,0.374581
1,31,Atlético-MG,Internacional,0.273077,0.341815,0.385107
2,31,Vitória,Fluminense,0.535688,0.232638,0.231674
3,31,Flamengo,Juventude,0.425212,0.329921,0.244867
4,31,Vasco,Bahia,0.533508,0.209729,0.256763


In [ ]:
probabilidades_jogos_restantes_RL.to_csv("drive/My Drive/Projeto Futebol/probabilidades_jogos_restantes_RegressãoLogística_rodada30.csv", index=False)